In [161]:
from openai import OpenAI
import pandas as pd
import json
from helper import get_openai_api_key
from typing import List, Dict, Any, Tuple
import pandas as pd
import json
import time
import re
import numpy as np


In [162]:
MODEL = "gpt-4.1-nano"
EMBEDDING_MODEL = "text-embedding-3-small"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100
TOP_K = 5
openai_api_key = get_openai_api_key()
client = OpenAI(api_key=openai_api_key)


In [163]:
def call_chat_model(system_prompt, user_prompt, model=MODEL, max_tokens=400):
    """
    Single-call wrapper to OpenAI chat completion (chat/completions style).
    Relies on a global `client` and default `MODEL`.
    """
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=max_tokens,
        temperature=0.0,
    )
    return resp.choices[0].message.content.strip()

In [164]:
def chunk_text(text: str, chunk_size: int = CHUNK_SIZE, overlap: int = CHUNK_OVERLAP) -> List[str]:
    """
    Split text into chunks of specified size with overlap."""
    if not text:
        return []
    chunks = []
    start = 0
    text_len = len(text)
    while start < text_len:
        end = min(start + chunk_size, text_len)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        if end == text_len:
            break
        start = end - overlap  # step forward with overlap
    return chunks

In [165]:
# --------- EMBEDDING / INDEX BUILD ----------
def embed_texts(texts, model=EMBEDDING_MODEL, batch_size=8, sleep_between=0.2):
    """
    Get embeddings for a list of texts. Uses batching to be gentle on the API.
    Returns list of vectors (numpy arrays).
    """
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        resp = client.embeddings.create(model=model, input=batch)
        for item in resp.data:
            vec = np.array(item.embedding, dtype=np.float32)
            embeddings.append(vec)
        time.sleep(sleep_between)
    return embeddings

In [166]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    """Anygle cosine similarity between two vectors. Used for RAG retrieval."""
    if a.ndim == 1:
        a = a.reshape(1, -1)
    if b.ndim == 1:
        b = b.reshape(1, -1)
    num = (a * b).sum()
    denom = (np.linalg.norm(a) * np.linalg.norm(b))
    return float(num / denom) if denom else 0.0

In [167]:
def build_index_from_df(df: pd.DataFrame):
    """
    Build an in-memory index of product chunks.
    Only indexing these columns: product_id, name, category, description, seasons, price.
    (Stock is deliberately excluded so that stock changes don’t require a re-embed.)

    Returns: list of dicts: {"embedding": ..., "row_idx": int, "chunk": str}
    """

    cols = [c for c in ["product_id", "name", "category", "description", "seasons", "price"] if c in df.columns]
    safe_df = df[cols].copy()

    row_chunks = [] 
    for idx, row in safe_df.iterrows():
        combined = (
            f"product_id: {row.get('product_id', '')}\n"
            f"name: {row.get('name', '')}\n"
            f"category: {row.get('category', '')}\n"
            f"description: {row.get('description', '')}\n"
            f"seasons: {row.get('seasons', '')}\n"
            f"price: {row.get('price', '')}"
        )
        for c in chunk_text(combined):
            row_chunks.append((idx, c))

    texts = [c for (_, c) in row_chunks]
    embeddings = embed_texts(texts) if texts else []
    index = []
    for emb, (row_idx, chunk_text_) in zip(embeddings, row_chunks):
        index.append({"embedding": emb, "row_idx": row_idx, "chunk": chunk_text_})
    return index

In [168]:
def search_index(query: str, index: List[Dict[str, Any]], top_k: int = TOP_K) -> Tuple[List[Tuple[float, Dict[str, Any]]], List[int]]:
    """Retrieve top-k chunks using cosine similarity"""
    if not query or not index:
        return [], []
    q_emb = embed_texts([query])[0]
    q_emb = np.array(q_emb)
    scored = []
    for item in index:
        sim = cosine_similarity(q_emb, np.array(item["embedding"]))
        scored.append((sim, item))
    scored.sort(key=lambda x: x[0], reverse=True)
    top = scored[:top_k]
    row_idxs = [item["row_idx"] for (_, item) in top]
    return top, row_idxs

In [169]:
def generate_augmented_prompt(query: str, retrieved: List[Tuple[float, Dict[str, Any]]]) -> str:
    """
    Building a system+user prompt where we provide retrieved context and ask the LLM to answer.
    """
    context_blocks = []
    for score, item in retrieved:
        context_blocks.append(f"[score={score:.3f}]\n{item['chunk']}")
    context_text = "\n\n".join(context_blocks) if context_blocks else ""
    user_prompt = (
        "You will be provided an user enquiry regarding products. Use ONLY the context below and do not assume anything.\n"
        "If the question is unrelated to the context, say that you don’t have enough info.\n\n"
        f"CONTEXT:\n{context_text}\n\n"
        f"QUESTION:\n{query}\n"
    )
    return user_prompt

In [170]:
def llm_classify_email(client, model: str, email: Dict[str, str]) -> str:

    system_prompt = (
        "You are an email intent classifier. Choose EXACTLY ONE label:\n"
        "• order request  – the sender intends to buy/place an order OR asks concrete purchasing details "
        "(price, quantity, availability/stock, lead time, shipping) with intent to proceed.\n"
        "• product inquiry – general questions (features, specs, comparisons, suitability) without intent to purchase now.\n"
        "Return ONLY the label text: 'order request' or 'product inquiry'. No punctuation, no JSON."
    )
    user_prompt = (
        f"EMAIL_ID: {email.get('email_id','')}\n"
        f"SUBJECT: {email.get('subject','')}\n"
        f"MESSAGE: {email.get('message','')}\n\n"
        "Answer with exactly one label."
    )

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.0,
    )
    text = (resp.choices[0].message.content or "").strip().lower()
    return text


In [171]:
def llm_extract_order_items(client, model: str, email: Dict[str, str]) -> List[Dict[str, Any]]:
    """
    Ask the LLM to extract product requests and quantities from a single order email.
    Returns list of dicts with EXACT keys:
      - product_id: string SKU/ID if explicitly present in the email, else null
      - product_name: human-readable name/descriptor from the email (non-empty string)
      - quantity: number of order integer >= 1 (default to 1 if unclear)
      - raw_text: exact substring copied from the email that mentions the item/quantity
    """
    import json

    system_prompt = (
        "You are a precise information extractor. Return ONLY a valid JSON array. "
        "Each array element is an object with EXACTLY these keys in any order: "
        "product_id, product_name, quantity, raw_text. "
        "Rules:\n"
        "- product_id: the exact SKU/ID if explicitly present in the email; otherwise null.\n"
        "- product_name: concise name/descriptor from the email (must be a non-empty string).\n"
        "- quantity: integer >= 1. Default to 1 if not clearly stated. Convert words like 'two' -> 2.\n"
        "- raw_text: copy the exact snippet from the email that mentions the item/quantity.\n"
        "Do NOT include extra keys. Do NOT include explanations, markdown, or code fences. "
        "If no items are found, return an empty JSON array []."
    )

    user_prompt = (
        "Extract product order lines from the following customer email.\n\n"
        f"EMAIL_ID: {email.get('email_id','')}\n"
        f"SUBJECT: {email.get('subject','')}\n"
        f"MESSAGE:\n{email.get('message','')}\n\n"
        "Return ONLY the JSON array."
    )

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.0,
    )
    text = (resp.choices[0].message.content or "").strip()

    print(f"LLM response for order items extraction from email_id={email.get('email_id','')}:\n{text}\n")

    try:
        items = json.loads(text)
        print(f"Extracted items from email_id={email.get('email_id','')}: {items}")
        return items if isinstance(items, list) else []
    except Exception:
        return []


In [172]:
def llm_generate_order_response(client, model: str, email_id: str, order_lines: List[Dict[str,Any]], products: pd.DataFrame) -> str:
    """Given processed order lines with status, ask LLM to craft a professional response email.
    order_lines: list of dicts with keys: product_id, product_name, requested_qty, status, allocated_qty(optional)
    """
    lines_summary = []
    for l in order_lines:
        pid = l.get('product_id') or l.get('product_name')
        status = l.get('status')
        rq = l.get('requested_qty')
        aq = l.get('allocated_qty', 0)
        pname = l.get('product_name')

        if status == 'created':
            lines_summary.append(f"{pname}: {aq} of {rq} — confirmed")
        elif status == 'out of stock':
            lines_summary.append(f"{pname}: requested {rq} — out of stock")
        else:
            lines_summary.append(f"{pname}: requested {rq} — status: {status}")
    summary_text = "\n".join(lines_summary)

    print(f"Order summary for email_id={email_id}:\n{summary_text}\n")

    prompt = (
        f"Write a professional customer-facing email in reply to order {email_id}. Use a concise, helpful tone.\n"
        f"Order summary:\n{summary_text}\n\n"
        "Properly format the email with subject greeting and signature."
        "if atleast one order is confirmed then Subject should be 'Order for Product Name(s) - Confirmed'. in body Mention that the order has been received and and confirmed and customer will be notified once shipped."
        "if all are out of stock then Subject should be 'COrder for Product Name(s) - Out of stock'. In body mention that the order cannot be placed due to unavailability of stock."
        "Greet with Dear Customer, in the body thank the customer, then provide the update, and end with 'Best regards \n Dibyendu from ShopOnline' ."
        "Tell them to reach out for any questions."
        "adapt tone appropriately based on the context of the customer's request"
        
    )
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role":"user","content":prompt}],
        temperature=0.2,
    )
    print(f"Generated order response for email_id={email_id}")
    return resp.choices[0].message.content.strip()

In [173]:
def process_order_requests_pipeline(client, model: str, emails_df: pd.DataFrame, classification_df: pd.DataFrame, products_df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Process a single email labeled as 'order request'. Returns (order_status_df, order_response_df)."""
    order_status_rows: List[Dict[str, Any]] = []
    order_responses: List[Dict[str, Any]] = []

    # Expect a single email row; merge to confirm it's an order request
    merged = emails_df.merge(classification_df, on='email_id', how='inner')
    if merged.empty or str(merged.iloc[0].get('category', '')).lower() != 'order request':
        # Nothing to do for non-order emails
        return (
            pd.DataFrame(columns=['email_id','product_id','quantity','status']),
            pd.DataFrame(columns=['email_id','response'])
        )

    email = merged.iloc[0].to_dict()

    # Extract items from this one email
    items = llm_extract_order_items(client, model, email)
    print(f"Extracted items from email_id={email.get('email_id','')}: {items}")
    processed_lines = []

    for it in (items or []):
        pid = it.get('product_id')
        pname = it.get('product_name')
        qty = int(it.get('quantity', 1) or 1)
        allocated = 0
        status = 'out of stock'

        # Try product_id exact match first
        if pid and 'product_id' in products_df.columns:
            prod_idx = products_df[products_df['product_id'] == pid]
            if not prod_idx.empty:
                stock = int(prod_idx.iloc[0].get('stock', 0) or 0)
                if stock >= qty:
                    allocated = qty
                    status = 'created'
                    products_df.loc[products_df['product_id'] == pid, 'stock'] = stock - qty
                elif stock > 0:
                    allocated = stock
                    status = 'out of stock'
                    products_df.loc[products_df['product_id'] == pid, 'stock'] = 0

        # Fallback by exact name match (case-insensitive)
        if allocated == 0 and pname and 'name' in products_df.columns:
            prod_idx = products_df[products_df['name'].str.lower() == str(pname).lower()]
            if not prod_idx.empty:
                pid = prod_idx.iloc[0].get('product_id', pid)
                stock = int(prod_idx.iloc[0].get('stock', 0) or 0)
                if stock >= qty:
                    allocated = qty
                    status = 'created'
                    products_df.loc[products_df['name'].str.lower() == str(pname).lower(), 'stock'] = stock - qty
                elif stock > 0:
                    allocated = stock
                    status = 'out of stock'
                    products_df.loc[products_df['name'].str.lower() == str(pname).lower(), 'stock'] = 0

        processed_lines.append({
            'product_id': pid,
            'product_name': pname,
            'requested_qty': qty,
            'allocated_qty': allocated,
            'status': status,
        })

        order_status_rows.append({
            'email_id': email.get('email_id'),
            'product_id': pid,
            'quantity': qty,
            'status': 'created' if allocated == qty and qty > 0 else 'out of stock',
        })

    # Generate a customer-facing response for this email (even if no items parsed)
    response_text = llm_generate_order_response(client, model, email.get('email_id'), processed_lines, products_df)
    order_responses.append({'email_id': email.get('email_id'), 'response': response_text})

    order_status_df = pd.DataFrame(order_status_rows, columns=['email_id','product_id','quantity','status']) if order_status_rows else pd.DataFrame(columns=['email_id','product_id','quantity','status'])
    order_response_df = pd.DataFrame(order_responses, columns=['email_id','response'])
    return order_status_df, order_response_df


In [174]:
def llm_write_stock_df_query(client, model: str, email: Dict[str, str], product_inventory: pd.DataFrame) -> Dict[str, Any]:
    """
    Extract product_id(s) from the inquiry and RETURN the current stock details directly.

    Returns:
      {
        "product_ids": [..],
        "stock_details": [{"product_id": "<id>", "stock": <int>}, ...]
      }
    """
    import json, re

    prompt = (
        "Read the following customer inquiry and extract any product_id values mentioned.\n"
        "Return ONLY JSON with key 'product_ids' (array of strings). No explanation.\n\n"
        f"EMAIL_ID: {email.get('email_id','')}\n"
        f"SUBJECT: {email.get('subject','')}\n"
        f"MESSAGE: {email.get('message','')}\n"
    )
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    text = resp.choices[0].message.content
    out = {"product_ids": [], "stock_details": []}

    # Parse product_ids from model output
    try:
        data = json.loads(text)
        if isinstance(data, dict):
            pids = data.get("product_ids") or []
            out["product_ids"] = [str(x) for x in pids if isinstance(x, (str, int))]
    except Exception:
        # If not valid JSON, try to scrape IDs that look like tokens with letters+digits/underscores
        # (kept minimal; you said no fallback classification logic, this is only for ID parsing if JSON slips)
        ids = re.findall(r"\b[A-Za-z0-9_-]{3,}\b", text or "")
        out["product_ids"] = ids

    # Build stock_details directly from the provided inventory
    if (
        isinstance(product_inventory, pd.DataFrame)
        and "product_id" in product_inventory.columns
        and "stock" in product_inventory.columns
        and out["product_ids"]
    ):
        sub = product_inventory.loc[
            product_inventory["product_id"].isin(out["product_ids"]),
            ["product_id", "stock"],
        ].copy()
        out["stock_details"] = [
            {"product_id": str(r.product_id), "stock": int(r.stock) if pd.notnull(r.stock) else 0}
            for _, r in sub.iterrows()
        ]

    print(
        f"Extracted product_ids {out['product_ids']} from email_id={email.get('email_id','')}; "
        f"stock_details={out['stock_details']}"
    )
    return out


In [175]:
def answer_inquiry_with_stock(
    client,
    model: str,
    products_df: pd.DataFrame,
    product_inventory: pd.DataFrame,
    email: Dict[str, str],
    top_k: int = TOP_K,
) -> str:
    # RAG retrieval from index
    query_txt = email.get("message") or email.get("subject") or ""
    retrieved, _ = search_index(query_txt, INDEX, top_k=top_k)

    # Extract product_ids and get live stock details directly
    stock_spec = llm_write_stock_df_query(client, model, email, product_inventory)
    stock_details = stock_spec.get("stock_details") or []

    # Build final prompt
    ctx_blocks = []
    for score, item in retrieved:
        ctx_blocks.append(f"[score={score:.3f}]\n{item['chunk']}")
    ctx_text = "\n\n".join(ctx_blocks) if ctx_blocks else ""

    stock_text = (
        "\n".join(f"{d.get('product_id')}: stock={int(d.get('stock', 0))}" for d in stock_details)
        if stock_details
        else "(no specific stock ids found)"
    )

    system_prompt = "You are a helpful product support assistant. Be precise and do not invent details."
    user_prompt = (
        "Use the product CONTEXT retrieved from Catalogue and the live STOCK info to write an reply email to the customer inquiry.\n"
        "Properly format the email with subject greeting body and signature."
        "Start with Dear Customer, and end with Best regards 'Dibyendu from ShopOnline' ."
        "If you lack stock info for an item they asked about, and if the customer has not explicitly asked for it, do not mention anything about stock. If it's available only then mention that it is available in stock. Do not quote how many stock you have, just mention available\n"
        "adapt tone appropriately based on the context of the customer's inquiry.\n"
        f"CONTEXT (retrieved RAG chunks):\n{ctx_text}\n\n"
        f"STOCK (current by product_id):\n{stock_text}\n\n"
        f"CUSTOMER EMAIL:\nID: {email.get('email_id','')}\nSUBJECT: {email.get('subject','')}\nMESSAGE: {email.get('message','')}\n"
    )
    print(f"Answering inquiry for email_id={email.get('email_id','')}")
    return call_chat_model(system_prompt, user_prompt)


In [176]:
def process_emails_one_by_one(client, model: str, emails_df: pd.DataFrame, products_df: pd.DataFrame,
                              index_df: pd.DataFrame = None,
                              save_paths: Dict[str,str] = None) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Process emails sequentially.

    Flow per email:
      1) llm_classify_email -> append to email-classification
      2) if 'order request':
            llm_extract_order_items -> process_order_requests_pipeline (updates inventory & order-status)
            -> llm_generate_order_response (append to order-response)
         else 'product inquiry':
            answer_inquiry_with_stock (RAG + stock by product_id) -> append to inquiry-response

    At the end: saves CSVs if `save_paths` provided.

    Returns: (product_inventory, classification_df, order_status_df, order_response_df, inquiry_response_df)
    """
    # Copy of products_df as product_inventory (mutable)
    product_inventory = products_df.copy(deep=True)

    # Build / reuse index (on allowed columns only)
    global INDEX
    base_for_index = index_df if index_df is not None else product_inventory
    INDEX = build_index_from_df(base_for_index)

    classification_rows: List[Dict[str, Any]] = []
    order_status_all: List[pd.DataFrame] = []
    order_response_rows: List[Dict[str, Any]] = []
    inquiry_response_rows: List[Dict[str, Any]] = []

    # Process one-by-one
    for _, row in emails_df.iterrows():
        email = row.to_dict()
        category = llm_classify_email(client, model, email)
        classification_rows.append({'email_id': email.get('email_id'), 'category': category})

        if category == 'order request':
            # Build a temp classification_df for the single email to reuse your existing pipeline
            tmp_class_df = pd.DataFrame([{'email_id': email.get('email_id'), 'category': 'order request'}])
            # Use the kept pipeline against the single email row
            single_email_df = pd.DataFrame([email])
            order_status_df, order_response_df = process_order_requests_pipeline(
                client, model, single_email_df, tmp_class_df, product_inventory
            )
            if not order_status_df.empty:
                order_status_all.append(order_status_df)
            if not order_response_df.empty:
                order_response_rows.extend(order_response_df.to_dict('records'))

        else:  # product inquiry
            response_text = answer_inquiry_with_stock(client, model, products_df, product_inventory, email)
            inquiry_response_rows.append({'email_id': email.get('email_id'), 'response': response_text})

    # Final DataFrames with required schemas
    classification_df = pd.DataFrame(classification_rows, columns=['email_id','category']) if classification_rows else pd.DataFrame(columns=['email_id','category'])
    order_status_df = pd.concat(order_status_all, ignore_index=True) if order_status_all else pd.DataFrame(columns=['email_id','product_id','quantity','status'])
    order_response_df = pd.DataFrame(order_response_rows, columns=['email_id','response']) if order_response_rows else pd.DataFrame(columns=['email_id','response'])
    inquiry_response_df = pd.DataFrame(inquiry_response_rows, columns=['email_id','response']) if inquiry_response_rows else pd.DataFrame(columns=['email_id','response'])

    # Save CSVs if requested
    if save_paths:
        if save_paths.get('classification'):
            classification_df.to_csv(save_paths['classification'], index=False)
        if save_paths.get('order_status'):
            order_status_df.to_csv(save_paths['order_status'], index=False)
        if save_paths.get('order_response'):
            order_response_df.to_csv(save_paths['order_response'], index=False)
        if save_paths.get('inquiry_response'):
            inquiry_response_df.to_csv(save_paths['inquiry_response'], index=False)

    return product_inventory, classification_df, order_status_df, order_response_df, inquiry_response_df


In [177]:
DEFAULT_SAVE_PATHS = {
    'classification': 'email-classification.csv',
    'order_status': 'order-status.csv',
    'order_response': 'order-response.csv',
    'inquiry_response': 'inquiry-response.csv',
    'udated_inventory': 'product_inventory.csv',
}

In [178]:
products_df = pd.read_csv('products.csv')
emails_df = pd.read_csv('emails.csv')

In [179]:
product_inventory, classification_df, order_status_df, order_response_df, inquiry_response_df = process_emails_one_by_one(
    client=client,
    model=MODEL,
    emails_df=emails_df.head(5),
    products_df=products_df,   # copied internally to product_inventory
    index_df=None,             # optional: pass a curated df for building the RAG index
    save_paths=DEFAULT_SAVE_PATHS      # writes the CSVs at the end
)

LLM response for order items extraction from email_id=E001:
[
  {
    "product_id": "LTH0976",
    "product_name": "Leather Bifold Wallets",
    "quantity": 1,
    "raw_text": "all the remaining LTH0976 Leather Bifold Wallets you have in stock"
  }
]

Extracted items from email_id=E001: [{'product_id': 'LTH0976', 'product_name': 'Leather Bifold Wallets', 'quantity': 1, 'raw_text': 'all the remaining LTH0976 Leather Bifold Wallets you have in stock'}]
Extracted items from email_id=E001: [{'product_id': 'LTH0976', 'product_name': 'Leather Bifold Wallets', 'quantity': 1, 'raw_text': 'all the remaining LTH0976 Leather Bifold Wallets you have in stock'}]
Order summary for email_id=E001:
Leather Bifold Wallets: 1 of 1 — confirmed

Generated order response for email_id=E001
LLM response for order items extraction from email_id=E002:
[
  {
    "product_id": "VBT2345",
    "product_name": "Vibrant Tote bag",
    "quantity": 1,
    "raw_text": "buy the VBT2345 Vibrant Tote bag"
  }
]

Extracted 